# Wien GTFS data to SQLITE database
<hr>

## What is GTFS?
A GTFS feed, which contains static transit information, is composed of a number of text (.txt) files that are contained in a single ZIP file.  
Each file describes a particular aspect of transit information: stops, routes, trips, fares, etc.  
source: https://gtfs.org/schedule/

Wien GTFS website  
https://www.data.gv.at/katalog/dataset/wiener-linien-fahrplandaten-gtfs-wien

SQLITE Documentation  
https://www.sqlite.org/docs.html

## Imports and useful functions

In [1]:
import csv
import os
import sqlite3
import wget #https://pypi.org/project/wget/

from pathlib import Path
from zipfile import ZipFile

USER = "xn"
DOWN_PATH = Path(f"/home/{USER}/Documents/wien_gtfs")

def create_dir(down_path):
    if os.path.exists(down_path):
        print(f"{down_path} Download Directory exists")
        pass
    else:
        try:
            os.mkdir(down_path)
            print(f"{down_path} download Directory created")
        except Exception as err:
            print(err)
            
create_dir(DOWN_PATH)

/home/xn/Documents/wien_gtfs Download Directory exists


## Download Wien GTFS data
from official site: transitfeeds.com/p/stadt-wien/888/latest/download

In [2]:
url = "https://transitfeeds.com/p/stadt-wien/888/latest/download"
file_name = str(DOWN_PATH/"wien_gtfs.zip")
print("Downloading latest GTFS data from transitfeeds.com/p/stadt-wien/888")
wget.download(url,out=file_name)

100% [........................................................................] 16881388 / 16881388

'/home/xn/Documents/wien_gtfs/wien_gtfs (1).zip'

## Extract zip GTFS file

In [3]:
IN_DIR = DOWN_PATH/"in_dir"
create_dir(IN_DIR)

with ZipFile(file_name, 'r') as zip_file:
    print(f"Extracting {file_name}")
    try:
        zip_file.extractall(IN_DIR)
    except Exception as err:
        print(err)

/home/xn/Documents/wien_gtfs/in_dir Download Directory exists
Extracting /home/xn/Documents/wien_gtfs/wien_gtfs.zip


## Define GTFS txt parsers

In [4]:
def parse_agency(IN_DIR, file_name):
    """
    agency_id,
    agency_name,
    agency_url,
    agency_timezone,
    agency_lang,
    agency_phone
    """
    with open(IN_DIR/file_name, 'r') as file:
        dr = csv.DictReader(file)
        agency = [(col["agency_id"], 
                  col["agency_name"],
                  col["agency_url"],
                  col["agency_timezone"],
                  col["agency_lang"],
                  col["agency_phone"]
                  )
                  for col in dr]
    return agency

def parse_calendar(IN_DIR, file_name):
    """
    service_id,
    monday,
    tuesday,
    wednesday,
    thursday,
    friday,
    saturday,
    sunday,
    start_date,
    end_date
    """
    with open(IN_DIR/file_name,'r') as file:
        dr = csv.DictReader(file)
        calendar = [(col["\ufeffservice_id"], 
                     col["monday"],
                     col["tuesday"],
                     col["wednesday"],
                     col["thursday"],
                     col["friday"],
                     col["saturday"],
                     col["sunday"],
                     col["start_date"],
                     col["end_date"]
                    )
                    for col in dr]
    return calendar

def parse_calendar_dates(IN_DIR, file_name):
    """
    service_id,
    date,
    exception_type
    """
    with open(IN_DIR/file_name,'r') as file:
        dr = csv.DictReader(file)
        calendar_dates = [(col["\ufeffservice_id"], 
                           col["date"],
                           col["exception_type"]
                          )
                          for col in dr]
    return calendar_dates
    
def parse_routes(IN_DIR, file_name):
    """
    route_id,
    agency_id,
    route_short_name,
    route_long_name,
    route_type,
    route_color,
    route_text_color
    """
    with open(IN_DIR/file_name,'r') as file:
        dr = csv.DictReader(file)
        routes = [(col["\ufeffroute_id"], 
                   col["agency_id"],
                   col["route_short_name"],
                   col["route_long_name"],
                   col["route_type"],
                   col["route_color"],
                   col["route_text_color"]
                  )
                  for col in dr]
    return routes
    
def parse_trips(IN_DIR, file_name):
    """
    route_id,
    service_id,
    trip_id,
    shape_id,
    trip_headsign,
    direction_id,
    block_id
    """
    with open(IN_DIR/file_name,'r') as file:
        dr = csv.DictReader(file)
        trips = [(col["\ufeffroute_id"], 
                  col["service_id"],
                  col["trip_id"],
                  col["shape_id"],
                  col["trip_headsign"],
                  col["direction_id"],
                  col["block_id"]
                 )
                 for col in dr]
    return trips

def parse_stop_times(IN_DIR, file_name):
    """
    trip_id,
    arrival_time,
    departure_time,
    stop_id,
    stop_sequence,
    pickup_type,
    drop_off_type,
    shape_dist_traveled
    """
    with open(IN_DIR/file_name,'r') as file:
        dr = csv.DictReader(file)
        stop_times = [(col["\ufefftrip_id"], 
                       col["arrival_time"],
                       col["departure_time"],
                       col["stop_id"],
                       col["stop_sequence"],      
                       col["pickup_type"],
                       col["drop_off_type"],
                       col["shape_dist_traveled"]
                      )
                      for col in dr]
    return stop_times

def parse_stops(IN_DIR, file_name):
    """
    stop_id,
    stop_name,
    stop_lat,
    stop_lon
    """
    with open(IN_DIR/"stops.txt",'r') as file:
        dr = csv.DictReader(file)
        stops = [(col["stop_id"], 
                  col["stop_name"],
                  col["stop_lat"],
                  col["stop_lon"])
                 for col in dr]
    return stops

def parse_shapes(IN_DIR, file_name):
    """
    shape_id,
    shape_pt_lat,
    shape_pt_lon,
    shape_pt_sequence,
    shape_dist_traveled
    """
    with open(IN_DIR/"shapes.txt",'r') as file:
        dr = csv.DictReader(file)
        shapes = [(col["\ufeffshape_id"], 
                  col["shape_pt_lat"],
                  col["shape_pt_lon"],
                  col["shape_pt_sequence"],
                  col["shape_dist_traveled"]
                  )
                  for col in dr]
    return shapes

#Todo: find a common implementation to parse based on supplied column names

In [5]:
%%time
agency_data = parse_agency(IN_DIR,"agency.txt")
calendar_data = parse_calendar(IN_DIR,"calendar.txt")
calendar_dates_data = parse_calendar_dates(IN_DIR,"calendar_dates.txt")
routes_data = parse_routes(IN_DIR,"routes.txt")
shapes_data = parse_shapes(IN_DIR,"shapes.txt")
stop_times_data = parse_stop_times(IN_DIR,"stop_times.txt")
stops_data = parse_stops(IN_DIR,"stops.txt")
trips_data = parse_trips(IN_DIR,"trips.txt")

CPU times: user 4.05 s, sys: 299 ms, total: 4.35 s
Wall time: 4.36 s


In [7]:
print(agency_data[0])
print(calendar_data[0])
print(calendar_dates_data[0])
print(routes_data[0])
print(shapes_data[0])
print(stop_times_data[0])
print(stops_data[0])
print(trips_data[0])

('03', 'WLB', 'https://www.wlb.at/', 'Europe/Vienna', 'DE', '+43 (0) 1/ 90 444')
('T0', '1', '1', '1', '1', '1', '0', '0', '20191215', '20201212')
('T0', '20191225', '2')
('11-WLB-j20-1', '03', 'WLB', 'Wien Oper - Wiener Neudorf - Guntramsdorf - Traiskirchen - Baden', '0', '0A295D', 'FFFFFF')
('11-WLB-j20-1.1.H', '48.2020032', '16.3706372', '1', '0.00')
('518.T0.11-WLB-j20-1.1.H', '19:12:00', '19:12:00', 'at:49:975:0:7', '1', '0', '0', '0.00')
('at:43:3121:0:1', 'Baden Josefsplatz', '48.0059514477605', '16.2336980467103')
('11-WLB-j20-1', 'T0+32', '1.T0.11-WLB-j20-1.14.R', '11-WLB-j20-1.14.R', 'Wien Aßmayergasse', '1', '')


## Create the wien_gtfs_db database

In [8]:
DB_DIR = DOWN_PATH/"db"
create_dir(DB_DIR)

print("Creating wien_gtfs_db.db")
DB_NAME = "wien_gtfs_db.db"
con = sqlite3.connect(DB_DIR/DB_NAME)
cursor = con.cursor()

/home/xn/Documents/wien_gtfs/db Download Directory exists
Creating wien_gtfs_db.db


## Create tables in database

Tables:
* agency   
* routes  
* calendar_dates  
* calendar    
* shapes  
* stops  
* stop_times  
* trips  

In [9]:
try:
    #agency
    cursor.execute("""CREATE TABLE IF NOT EXISTS agency 
    (
    agency_id INTEGER NOT NULL PRIMARY KEY, 
    agency_data TEXT,
    agency_name TEXT, 
    agency_url TEXT,
    agency_timezone TEXT, 
    agency_lang TEXT, 
    agency_phone TEXT
    );
    """
                     )
    
    #routes
    cursor.execute("""CREATE TABLE IF NOT EXISTS routes 
    (
    route_id TEXT NOT NULL PRIMARY KEY, 
    agency_id TEXT NOT NULL,
    route_short_name TEXT,
    route_long_name TEXT,
    route_type INTEGER,
    route_color TEXT,
    route_text_color TEXT
    );
    """
                     )
    
    #calendar
    cursor.execute("""CREATE TABLE IF NOT EXISTS calendar
    (
    service_id TEXT NOT NULL PRIMARY KEY,
    monday INTEGER,
    tuesday INTEGER,
    wednesday INTEGER,
    thursday INTEGER,
    friday INTEGER,
    saturday INTEGER,
    sunday INTEGER,
    start_date TEXT,
    end_date TEXT
    );
    """
                     )
    
    #calendar_dates
    cursor.execute("""CREATE TABLE IF NOT EXISTS calendar_dates 
    (
    service_id TEXT NOT NULL,
    date TEXT,
    exception_type INTEGER
    );
    """
                     )
    
   #shapes
    cursor.execute("""CREATE TABLE IF NOT EXISTS shapes 
    (
    shape_id TEXT NOT NULL,
    shape_pt_lat REAL,
    shape_pt_lon REAL,
    shape_pt_sequence INTEGER,
    shape_dist_traveled REAL
    );
    """
                     )
    
    #stops
    cursor.execute("""CREATE TABLE IF NOT EXISTS stops 
    (
    stop_id TEXT NOT NULL PRIMARY KEY,
    stop_name TEXT,
    stop_lat REAL,
    stop_lon REAL
    );
    """
                     )
    
    #stop_times
    cursor.execute("""CREATE TABLE IF NOT EXISTS stop_times 
    (
    trip_id TEXT NOT NULL,
    arrival_time TEXT,
    departure_time TEXT,
    stop_id TEXT,
    stop_sequence INTEGER,
    pickup_type INTEGER,
    drop_off_type INTEGER,
    shape_dist_traveled REAL
    );
    """
                     )
    
    #trips
    cursor.execute("""CREATE TABLE IF NOT EXISTS trips 
    (
    route_id TEXT NOT NULL,
    service_id TEXT,
    trip_id TEXT NOT NULL,
    shape_id TEXT NOT NULL,
    trip_headsign TEXT,
    direction_id TEXT,
    block_id INTEGER
    );
    """
                     )

except (sqlite3.OperationalError, IntegrityError) as err:
    print (err)
    print("rolling back table creation...")
    con.rollback()

In [10]:
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
print(cursor.fetchall())

[('agency',), ('routes',), ('calendar',), ('calendar_dates',), ('shapes',), ('stops',), ('stop_times',), ('trips',)]


## Populate tables with GTFS rows

In [11]:
%%time
try:
    #Agency
    cursor.execute("DELETE FROM agency")
    print("inserting into agency...")
    cursor.executemany("""INSERT INTO agency
    (
    agency_id, 
    agency_name, 
    agency_url, 
    agency_timezone,
    agency_lang,
    agency_phone
    )
    VALUES (?,?,?,?,?,?);
    """,agency_data)
    
    cursor.execute("DELETE FROM calendar")
    print("inserting into calendar...")
    cursor.executemany("""INSERT INTO calendar
    (
    service_id,
    monday,
    tuesday,
    wednesday,
    thursday,
    friday,
    saturday,
    sunday,
    start_date,
    end_date
    )
    VALUES (?,?,?,?,?,?,?,?,?,?);""", 
                       calendar_data)
    con.commit()
    
    cursor.execute("DELETE FROM calendar_dates")
    print("inserting into calendar_dates...")
    cursor.executemany("""INSERT INTO calendar_dates
    (
    service_id,
    date,
    exception_type
    )
    VALUES (?,?,?);""", 
                       calendar_dates_data)
    con.commit()
    
    cursor.execute("DELETE FROM routes")
    print("inserting into routes...")
    cursor.executemany("""INSERT INTO routes 
    (
    route_id, 
    agency_id, 
    route_short_name,
    route_long_name,
    route_type,
    route_color,
    route_text_color) 
    VALUES (?,?,?,?,?,?,?);""", 
                       routes_data)
    
    cursor.execute("DELETE FROM trips")
    print("inserting into trips...")
    cursor.executemany("""INSERT INTO trips 
    (
    route_id, 
    service_id, 
    trip_id,
    shape_id,
    trip_headsign,
    direction_id,
    block_id
    )
    VALUES (?,?,?,?,?,?,?);""", 
                       trips_data)
    con.commit()
    
    cursor.execute("DELETE FROM stop_times")
    print("inserting into stop_times...")
    cursor.executemany("""INSERT INTO stop_times 
    (
    trip_id,
    arrival_time,
    departure_time,
    stop_id,
    stop_sequence,
    pickup_type,
    drop_off_type,
    shape_dist_traveled
    )
    VALUES (?,?,?,?,?,?,?,?);""", 
                       stop_times_data)
    con.commit()
    
    cursor.execute("DELETE FROM stops")
    print("inserting into stops...")
    cursor.executemany("""INSERT INTO stops 
    (
    stop_id,
    stop_name,
    stop_lat,
    stop_lon
    )
    VALUES (?,?,?,?);""", 
                       stops_data)
    con.commit()
    
    cursor.execute("DELETE FROM shapes")
    print("inserting into shapes...")    
    cursor.executemany("""INSERT INTO shapes
    (
    shape_id,
    shape_pt_lat,
    shape_pt_lon,
    shape_pt_sequence,
    shape_dist_traveled
    ) 
    VALUES (?,?,?,?,?);""", 
                       shapes_data)
    con.commit()
    
    
    
except sqlite3.OperationalError as OE:
    print (OE)
    print("rolling back table insertion...")
    con.rollback()

con.commit()
con.close()

inserting into agency...
inserting into calendar...
inserting into calendar_dates...
inserting into routes...
inserting into trips...
inserting into stop_times...
inserting into stops...
inserting into shapes...
CPU times: user 2.98 s, sys: 616 ms, total: 3.6 s
Wall time: 5.31 s
